In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import binarize
from sklearn.preprocessing import Binarizer
import matplotlib.pyplot as plt

from datetime import datetime

In [3]:
df=pd.read_excel('result_data.xlsx', skiprows=1)
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    38309 non-null  datetime64[ns]
 1   노출(분)   21525 non-null  float64       
 2   마더코드    38309 non-null  int64         
 3   상품코드    38309 non-null  int64         
 4   상품명     38309 non-null  object        
 5   상품군     38309 non-null  object        
 6   판매단가    38309 non-null  int64         
 7   취급액     35379 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [5]:
#1. 무형, 보험 상품 및 판매가 0인 프로그램은 제외
df=df[df['상품군']!='무형']

In [6]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 38303
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    37372 non-null  datetime64[ns]
 1   노출(분)   20588 non-null  float64       
 2   마더코드    37372 non-null  int64         
 3   상품코드    37372 non-null  int64         
 4   상품명     37372 non-null  object        
 5   상품군     37372 non-null  object        
 6   판매단가    37372 non-null  int64         
 7   취급액     35379 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.6+ MB


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


In [7]:
# 2. 취급액이 NaN 인경우 0으로 채워주기: 팔리지 않음
#    노출분이 NaN 인경우 ffill로 채워주기: 동일 상품 동일 노출(분)
df['노출(분)'].fillna(method='ffill', inplace=True)

/Users/jinwon-kim/Developer/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [8]:
df.fillna(0, inplace=True) #취급액이 NaN 인경우 0으로 채워주기: 팔리지 않음 (NaN인 열은 취급액 열 밖에 없으르모 한번에 채움)

/Users/jinwon-kim/Developer/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [9]:
df.info() #모든 결측값 제거 완료

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 38303
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    37372 non-null  datetime64[ns]
 1   노출(분)   37372 non-null  float64       
 2   마더코드    37372 non-null  int64         
 3   상품코드    37372 non-null  int64         
 4   상품명     37372 non-null  object        
 5   상품군     37372 non-null  object        
 6   판매단가    37372 non-null  int64         
 7   취급액     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.6+ MB


In [10]:
# 3. 주문량 = 취급액/판매단가(열 생성 하기)
df['주문량']=df['취급액']/df['판매단가']

<ipython-input-10-32ba06b44df1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주문량']=df['취급액']/df['판매단가']


In [11]:
df.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045


In [12]:
# 4. 심야 2:00 ~ 6:20 시간대 제외
#    요일 열 새로 생성 0~6 월~일요일, 중 토요일 (18:00~18:20 제외)

df['요일']=df['방송일시'].dt.dayofweek

<ipython-input-12-d4ee20848b0e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['요일']=df['방송일시'].dt.dayofweek


In [13]:
df.index = df['방송일시']

In [14]:
df[df['요일']==5].between_time('17:00', '18:21') #토요일 18:00~18:20 은 이미 포함되지 않은 상태를 확인

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량,요일
방송일시,,,,,,,,,,
2019-01-05 17:00:00,2019-01-05 17:00:00,20.0,100435,201350,우리바다 손질왕꼬막 24팩,농수축,39900,52050000.0,1304.511278,5
2019-01-05 17:20:00,2019-01-05 17:20:00,20.0,100435,201350,우리바다 손질왕꼬막 24팩,농수축,39900,77249000.0,1936.065163,5
2019-01-05 17:40:00,2019-01-05 17:40:00,20.0,100435,201350,우리바다 손질왕꼬막 24팩,농수축,39900,104552000.0,2620.350877,5
2019-01-05 18:20:00,2019-01-05 18:20:00,20.0,100801,202365,바다먹자 국내산 손질꽃게 7팩,농수축,59900,26816000.0,447.679466,5
2019-01-12 17:00:00,2019-01-12 17:00:00,20.0,100491,201483,피시원 국내산 손질 대구 8팩 * 매운탕 양념 8팩,농수축,59900,22580000.0,376.961603,5
...,...,...,...,...,...,...,...,...,...,...
2019-12-21 17:20:00,2019-12-21 17:20:00,20.0,100418,201322,코몽트 덤블 플리스 자켓 2종,의류,39900,32891000.0,824.335840,5
2019-12-21 17:40:00,2019-12-21 17:40:00,20.0,100418,201322,코몽트 덤블 플리스 자켓 2종,의류,39900,223492000.0,5601.303258,5
2019-12-28 17:00:00,2019-12-28 17:00:00,20.0,100202,201018,임성근의 녹용도가니탕 풀세트,농수축,55900,56234000.0,1005.974955,5


In [15]:
# 심야 2:00 ~ 6:20 시간대 제외
df.between_time('2:00','6:19')

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량,요일
방송일시,,,,,,,,,,
2019-01-01 06:00:00,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516,1
2019-01-01 06:00:00,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872,1
2019-01-02 06:00:00,2019-01-02 06:00:00,20.0,100229,200800,벨라홈 논슬립 옷걸이 세트,생활용품,29800,9605000.0,322.315436,2
2019-01-03 06:00:00,2019-01-03 06:00:00,20.0,100781,202292,에펨 여성 미들퍼부츠,잡화,39800,2450000.0,61.557789,3
2019-01-03 06:00:00,2019-01-03 06:00:00,20.0,100781,202285,에펨 남성 미들퍼부츠,잡화,49800,2645000.0,53.112450,3
...,...,...,...,...,...,...,...,...,...,...
2019-12-26 02:00:00,2019-12-26 02:00:00,20.0,100624,201927,안국루테인 미니100(5박스),건강기능,59000,18822000.0,319.016949,3
2019-12-27 02:00:00,2019-12-27 02:00:00,20.0,100312,201006,블랙모어스 알티지 오메가3 3병 (53주분),건강기능,95200,16867000.0,177.174370,4
2019-12-28 02:00:00,2019-12-28 02:00:00,20.0,100841,202484,옛날 그 쥐포 110장 (11팩),농수축,32900,33231000.0,1010.060790,5


In [16]:
df=df.drop(df.between_time('2:00','6:19').index)

In [146]:
df.to_csv('result4Analysis', index=False)

In [17]:
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량,요일
방송일시,,,,,,,,,,
2019-01-01 06:20:00,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386,1
2019-01-01 06:20:00,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777,1
2019-01-01 06:40:00,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045,1
2019-01-01 06:40:00,2019-01-01 06:40:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0,234.010025,1
2019-01-01 07:00:00,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,115.576271,1
...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:40:00,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,68.628378,1
2020-01-01 00:00:00,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,286.117978,2
2020-01-01 00:00:00,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,621.380952,2


In [ ]:
# groupby 상품군으로 분석해보기